In [2]:
# !git clone https://github.com/osamaizhar/Rag-pipelines-experiments

Cloning into 'Rag-pipelines-experiments'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 61 (delta 10), reused 7 (delta 7), pack-reused 47 (from 1)
Receiving objects: 100% (61/61), 43.69 MiB | 38.83 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [ ]:
# import os
# import pandas as pd
# import torch
# from transformers import AutoModel, AutoTokenizer

# class ExcelParserWithEmbeddings:
#     def __init__(self, directory):
#         self.directory = directory
#         self.excel_files = self.find_excel_files()

#         # Load embedding model
#         model_name = "jinaai/jina-embeddings-v2-base-en"
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#         self.embedding_model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

#     def find_excel_files(self):
#         """Search for all Excel files in the directory."""
#         return [f for f in os.listdir(self.directory) if f.endswith((".xlsx", ".xls"))]

#     def get_sheet_names(self, file_path):
#         """Returns a list of sheet names in an Excel file."""
#         try:
#             xls = pd.ExcelFile(file_path)
#             return xls.sheet_names
#         except Exception as e:
#             print(f"Error loading Excel file '{file_path}': {e}")
#             return []

#     def read_sheet(self, file_path, sheet_name):
#         """Reads a specific sheet into a DataFrame."""
#         try:
#             df = pd.read_excel(file_path, sheet_name=sheet_name)
#             df.fillna("", inplace=True)  # Replace NaNs with empty strings
#             return df
#         except Exception as e:
#             print(f"Error reading sheet '{sheet_name}' in '{file_path}': {e}")
#             return None

#     def get_embedding(self, text):
#         """Generates an embedding for a given text."""
#         inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
#         with torch.no_grad():
#             embeddings = self.embedding_model(**inputs).last_hidden_state.mean(dim=1)  # Mean pooling
#         return embeddings.squeeze().tolist()  # Convert tensor to list

#     def parse_and_embed(self):
#         """Parses all Excel files and generates embeddings for text data."""
#         parsed_data = {}

#         for file in self.excel_files:
#             file_path = os.path.join(self.directory, file)
#             parsed_data[file] = {}

#             for sheet in self.get_sheet_names(file_path):
#                 df = self.read_sheet(file_path, sheet)
#                 if df is None:
#                     continue

#                 sheet_embeddings = []
#                 for _, row in df.iterrows():
#                     row_text = " ".join(map(str, row.values))  # Concatenate row values as a single text string
#                     embedding = self.get_embedding(row_text)
#                     sheet_embeddings.append(embedding)

#                 parsed_data[file][sheet] = sheet_embeddings  # Store embeddings

#         return parsed_data

# # Usage Example
# # if __name__ == "__main__":


In [ ]:
# directory = "/kaggle/working/Rag-pipelines-experiments/data"  # Change to your actual directory
# parser = ExcelParserWithEmbeddings(directory)

# print(f"Found Excel files: {parser.excel_files}")

# embeddings_data = parser.parse_and_embed()


# # Print example embeddings for first file and sheet
# for file, sheets in embeddings_data.items():
#     print(f"\n📁 {file}:")
#     for sheet, embeddings in sheets.items():
#         print(f"  📄 Sheet: {sheet} -> {len(embeddings)} embeddings")
#         print(f"  🔹 First Embedding: {embeddings[0][:5]}...")  # Show first 5 values of first embedding
#         # break  # Only show one sheet per file for display
#     # break  # Only show one file for display

## EXCEL PARSER (".xlsx", ".xls")

In [12]:
import os
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer

def parse_excel_and_generate_embeddings(directory):
    """Parses all Excel files in a directory and generates embeddings for each row."""
    
    # Load Jina Embeddings Model
    model_name = "jinaai/jina-embeddings-v2-base-en"
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    embedding_model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

    # Find all Excel files
    excel_files = [f for f in os.listdir(directory) if f.endswith((".xlsx", ".xls"))]
    parsed_data = {}

    for file in excel_files:
        file_path = os.path.join(directory, file)
        parsed_data[file] = {}

        # Get all sheet names
        try:
            xls = pd.ExcelFile(file_path)
            sheet_names = xls.sheet_names
        except Exception as e:
            print(f"Error loading Excel file '{file}': {e}")
            continue

        for sheet in sheet_names:
            try:
                df = pd.read_excel(file_path, sheet_name=sheet)
                df.fillna("", inplace=True)  # Replace NaNs with empty strings

                # Generate embeddings
                sheet_embeddings = []
                for _, row in df.iterrows():
                    row_text = " ".join(map(str, row.values))  # Combine all column values
                    inputs = tokenizer(row_text, return_tensors="pt", padding=True, truncation=True)
                    with torch.no_grad():
                        embedding = embedding_model(**inputs).last_hidden_state.mean(dim=1)  # Mean pooling
                    sheet_embeddings.append(embedding.squeeze().tolist())

                parsed_data[file][sheet] = sheet_embeddings

            except Exception as e:
                print(f"Error reading sheet '{sheet}' in '{file}': {e}")
                continue
    # Print example embeddings for first file and sheet
    for file, sheets in parsed_data.items():
        print(f"\n📁 {file}:")
        for sheet, embeddings in sheets.items():
            print(f"  📄 Sheet: {sheet} -> {len(embeddings)} embeddings")
            print(f"  🔹 First Embedding: {embeddings[0][:5]}...")  # Show first 5 values of first embedding
            # break  # Only show one sheet per file for display
        # break  # Only show one file for display
    print("---\n\n\n\n")
    return sheet_embeddings




In [16]:
# Usage Example
directory = "/kaggle/working/Rag-pipelines-experiments/data"  # Change to your actual directory
excel_embeddings = parse_excel_and_generate_embeddings(directory)

<ipython-input-12-be8610c96230>:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)  # Replace NaNs with empty strings
<ipython-input-12-be8610c96230>:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)  # Replace NaNs with empty strings
<ipython-input-12-be8610c96230>:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)  # Replace NaNs with empty strings
<ipython-input-12-be8610c96230>:33: F


📁 Lesson 03 - Quiz.xlsx:
  📄 Sheet: Question -> 10 embeddings
  🔹 First Embedding: [0.6706684231758118, -0.30287522077560425, 1.0097568035125732, 0.1368766576051712, -0.3192911148071289]...

📁 Lesson 05 - Quiz.xlsx:
  📄 Sheet: Question -> 10 embeddings
  🔹 First Embedding: [-0.11480334401130676, -0.23468199372291565, 0.7329249978065491, 0.5030810236930847, 0.39307859539985657]...

📁 Lesson 04 - Quiz.xlsx:
  📄 Sheet: Question -> 10 embeddings
  🔹 First Embedding: [0.09186916798353195, -0.5301943421363831, 0.8090844750404358, 0.23199008405208588, -0.2556276321411133]...

📁 Lesson 01 - Quiz.xlsx:
  📄 Sheet: Question -> 10 embeddings
  🔹 First Embedding: [0.3007318079471588, -0.2416309118270874, 0.45901989936828613, -0.13181091845035553, -0.3009536862373352]...

📁 Exam 01.xlsx:
  📄 Sheet: Sheet1 -> 40 embeddings
  🔹 First Embedding: [0.2258109450340271, -0.785347044467926, 0.6225638389587402, -0.24165137112140656, -0.18876120448112488]...

📁 Lesson 02 - Quiz.xlsx:
  📄 Sheet: Question -> 1

In [33]:
# excel_embeddings
excel_embeddings[0]

[0.3087625205516815,
 0.003515701275318861,
 0.7871584296226501,
 -0.009421957656741142,
 -0.08734960108995438,
 0.6832818984985352,
 0.7424045205116272,
 -0.3710806667804718,
 0.8790549039840698,
 1.1079217195510864,
 -0.8256645202636719,
 -0.27679863572120667,
 -0.3299467861652374,
 0.6991616487503052,
 -0.38510861992836,
 1.002325177192688,
 0.11674291640520096,
 0.7229136824607849,
 0.1868438720703125,
 -0.46352148056030273,
 -0.11146179586648941,
 -0.3082284927368164,
 -0.33969491720199585,
 -0.5680312514305115,
 0.4655614495277405,
 0.8924027681350708,
 1.0586693286895752,
 0.40778031945228577,
 1.3023310899734497,
 0.5195680856704712,
 0.055996231734752655,
 0.07588140666484833,
 -0.17880305647850037,
 -0.3144116699695587,
 -0.1922241896390915,
 -0.8006397485733032,
 -0.27813592553138733,
 -0.043724603950977325,
 0.30106687545776367,
 0.8591137528419495,
 -0.4932015538215637,
 -0.0992218405008316,
 -0.3873176574707031,
 0.8048234581947327,
 0.07711752504110336,
 -0.4692027866840